In [50]:
import datasets 

from datasets import load_dataset

datasets = load_dataset('json',data_files='simplified-nq-train.jsonl')

In [51]:
print(datasets)


DatasetDict({
    train: Dataset({
        features: ['document_text', 'long_answer_candidates', 'question_text', 'annotations', 'document_url', 'example_id'],
        num_rows: 307373
    })
})


In [52]:
pip install transformers tensorRT

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [53]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

In [ ]:
pip install sacremoses

In [55]:
from transformers import pipeline
import pandas as pd
pipe = pipeline("translation", model="Helsinki-NLP/opus-mt-tc-big-en-pt")

In [56]:

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
  
tokenizer = AutoTokenizer.from_pretrained("unicamp-dl/translation-en-pt-t5")

model = AutoModelForSeq2SeqLM.from_pretrained("unicamp-dl/translation-en-pt-t5")

enpt_pipeline = pipeline('text2text-generation', model=model, tokenizer=tokenizer)

In [57]:
print(enpt_pipeline("translate English to Portuguese: He has been to Hawaii several times"))
print(pipe(">>por<< Tom tried to stab me."))


[{'generated_text': 'Foi várias vezes a Hawaii'}]
[{'translation_text': 'O Tom tentou esfaquear-me.'}]


In [58]:
def extrair_texto_da_lista(lista):
    if isinstance(lista, list):
        textos = []
        for elemento in lista:
            texto = elemento.get('generated_text', '')
            textos.append(texto)
        return textos
    return None


extrair_texto_da_lista(enpt_pipeline("translate English to Portuguese: He has been to Hawaii several times"))

['Foi várias vezes a Hawaii']

In [59]:
sample = datasets['train'][1]

In [60]:
small_set = datasets['train']

In [84]:
df = pd.DataFrame(columns = ['title', 'pergunta', 'answer'])

In [85]:
import re
import html

def clean_html_tags(string_list):
    cleaned_list = []
    for text in string_list:
        # Remove as tags HTML e decodifica entidades HTML
        cleaned_text = html.unescape(re.sub(r'<.*?>', '', text))
        cleaned_list.append(cleaned_text)
    return cleaned_list

def cleann_dict_list(dict_list):
    cleaned_list = []
    for dictionary in dict_list:
        if isinstance(dictionary, dict):
            cleaned_dict = {}
            for key, value in dictionary.items():
                if isinstance(value, str):
                    # Aplicar a limpeza de tags HTML
                    cleaned_value = clean_html_tags(value)
                    # Aplicar a limpeza de caracteres não alfanuméricos
                    cleaned_value = re.sub('[^A-Za-z0-9]', ' ', cleaned_value)
                    cleaned_dict[key] = cleaned_value
                else:
                    cleaned_dict[key] = value
            cleaned_list.append(cleaned_dict)
        else:
            cleaned_list.append(dictionary)
    return cleaned_list

In [86]:
for i in range(15):
    doc = small_set[i]['document_text']
    init = small_set[i]['annotations'][0]['long_answer']['start_token']
    end = small_set[i]['annotations'][0]['long_answer']['end_token']
    question = small_set[i]['question_text']
    
    # Extraindo a resposta
    answer = " ".join(doc.split(" ")[init:end])
    answer = enpt_pipeline('translate English to Portuguese: ' + answer)
    answer = extrair_texto_da_lista(answer)
    answer = clean_html_tags(answer)
    
    # Processando a pergunta
    question = enpt_pipeline('translate English to Portuguese: ' + question)
    question = extrair_texto_da_lista(question)
    question = clean_html_tags(question)
    
    new_row = {'title': [doc], 'pergunta': [question], 'answer': [answer]}
    df_aux = pd.DataFrame(new_row)
    df = pd.concat([df, df_aux])
df


,title,pergunta,answer
0,Email marketing - Wikipedia <H1> Email marketi...,[que é a utilização mais comum do marketing op...,[ Um exemplo comum de marketing de licença é u...
0,The Mother ( How I Met Your Mother ) - wikiped...,[como eu conheci a mãe que é a mãe],"[ Tracy McConnell , mais conhecido como ``A mãe]"
0,Human fertilization - wikipedia <H1> Human fer...,[o tipo de fertilização ocorre nos seres humanos],[ O processo de fertilização envolve a fusão d...
0,List of National Football League career quarte...,[que obteve a maioria dos vencedores no nfl],[ Active quarterback Tom Brady detém os regist...
0,Roanoke Colony - wikipedia <H1> Roanoke Colony...,[o que aconteceu com o assentamento perdido de...,[án]
0,List of regions of Africa - wikipedia <H1> Lis...,[quais as diferentes regiões da áfrica e como ...,[án]
0,Pom Klementieff - wikipedia <H1> Pom Klementie...,[que jogava mantis guardiãs da galáxia 2],[ Pom Klementieff (nascido em 3 de Maio de 1986)]
0,Frosty the Snowman ( film ) - wikipedia <H1> F...,[que fez a voz do médico em gelo o homem de neve],[án]
0,History of the Acadians - wikipedia <H1> Histo...,[o que a tribo indígena fez com que os acadian...,[ A sobrevivência dos assentamentos acadianos ...
0,Outer Banks - wikipedia <H1> Outer Banks </H1>...,[o que se considera os bancos externos da caró...,[án]
